In [1]:
# +
import warnings

# Essentials
import os
import numpy as np
import scipy.io as sio
import numpy as np
from tqdm import tqdm
import random

# Stain normalization
import stain_norm.stain_utils as utils
import stain_norm.vahadane as vahadane

# Image functions
from PIL import Image
from scipy.ndimage.morphology import binary_fill_holes
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.morphology import binary_closing, binary_dilation, disk
import random

In [2]:
# Parameters
IMPATH = '/home/as3ek/data/lizard/images/all_images/'
LBLPATH = '/home/as3ek/data/lizard/labels/Labels/'
patch_size = 128
resize_to = 128
target_images_path = '/home/as3ek/data/lizard_split_norm_bright3/images/' # for unnormalized patches
target_classes_path = '/home/as3ek/data/lizard_split_norm_bright3/classes/' # for unnormalized patches
target_instances_path = '/home/as3ek/data/lizard_split_norm_bright3/instances/' # for unnormalized patches
save_WSI = False
overlap = 0.5 # %-age area
random.seed(42)

# Create splits
test_percent = 7.5
splits = ["train", "test"]
for splt in splits:
    os.makedirs(os.path.join(target_images_path, splt), exist_ok=True)
    os.makedirs(os.path.join(target_classes_path, splt), exist_ok=True)
    os.makedirs(os.path.join(target_instances_path, splt), exist_ok=True)

In [3]:
def get_maps(label_pth):
    # Load label
    label = sio.loadmat(label_pth) #! This filename is a placeholder!
    # Load the instance segmentation map.
    # This map is of type int32 and contains values from 0 to N, where 0 is background
    # and N is the number of nuclei. 
    # Shape: (H, W) where H and W is the height and width of the image.
    inst_map = label['inst_map'] 

    # Load the index array. This determines the mapping between the nuclei in the instance map and the
    # corresponing provided categories, bounding boxes and centroids.
    nuclei_id = label['id'] # shape (N, 1), where N is the number of nuclei.

    # Load the nuclear categories / classes. 
    # Shape: (N, 1), where N is the number of nuclei.
    classes = label['class']
    
    # Map neucli to classes by id
    id_class_map = {nuclei_id[i][0]: classes[i][0] for i in range(len(nuclei_id))}
    
    # Replace id in instance map by class id
    class_map = np.copy(inst_map)
    for k, v in id_class_map.items():
        class_map[inst_map==k] = v
        
    # Convert to PIL images
    inst_pil = Image.fromarray(inst_map)
    class_pil = Image.fromarray(class_map)
    
    return inst_pil, class_pil

In [4]:
# Initialize normalizer
target = utils.read_image('/scratch/as3ek/datasets/lizard/images/all_images/dpath_11.png')
normalizer = vahadane.Normalizer()
normalizer.fit(target)

In [5]:
num_files = len(os.listdir(IMPATH))

for i, file in tqdm(enumerate(list(reversed(os.listdir(IMPATH))))):
    img_path = os.path.join(IMPATH, file)
    filename = file.split('.')[0]
    label_path = os.path.join(LBLPATH, filename)
    
    # image = Image.open(img_path).convert("RGB")
    image = utils.read_image(img_path)
    inst, clz = get_maps(label_path)
    
    # Stain normalize image
    image = normalizer.transform(image)
    # Normalize brightness
    image = utils.standardize_brightness(image)
    image = Image.fromarray(image)
    
    # Initialize x and y coord
    x_cord = 0
    y_cord = 0
    
    # For last row and column
    last_x = False
    last_y = False
    
    # Decide split for this image
    r = random.randint(0,100)
    if r <= test_percent:
        split = "test"
    else:
        split = "train"
    
    while x_cord + patch_size <= image.size[0]:
        while y_cord + patch_size <= image.size[1]:
            
            # Crop Image
            patch = image.crop((x_cord, y_cord, x_cord + patch_size, y_cord + patch_size))
            patch = patch.convert('RGB')
            patch = patch.resize((resize_to, resize_to))
            
            # Crop Instance and Class map
            inst_patch = inst.crop((x_cord, y_cord, x_cord + patch_size, y_cord + patch_size))
            clz_patch = clz.crop((x_cord, y_cord, x_cord + patch_size, y_cord + patch_size))
            
            # Generate filename
            filename = file.split('.')[0] + '__' + str(x_cord) + '_' + str(y_cord) + '.png'
            
            # Save image, class, instance
            patch.save(os.path.join(target_images_path, split, filename))
            clz_patch.save(os.path.join(target_classes_path, split, filename))
            inst_patch.save(os.path.join(target_instances_path, split, filename))
                        
            # Taking care of overlap
            y_cord = int(y_cord + (1 - overlap) * patch_size)
            
            # If next window is over the edge, re-fit window and set last flag
            if y_cord + patch_size > image.size[1]:
                if last_y == False:
                    y_cord = image.size[1] - patch_size
                    last_y = True
        
        # Taking care of overlap
        x_cord = int(x_cord + (1 - overlap) * patch_size)
        # Adjust for last row
        if x_cord + patch_size > image.size[0]:
            if last_x == False:
                x_cord = image.size[0] - patch_size
                last_x = True
        # Reset y-coordinate back to 0
        y_cord = 0

238it [35:57, 13.52s/it]


In [6]:
with open('/home/as3ek/github/histofusion/done_norm_bright2.txt', 'w') as f:
    f.write('Finished Patching!')